## 导入库

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## 加载原始数据集

In [2]:
# 明确指定有问题的列的数据类型（例如全部作为字符串读取）
dtype_dict = {7: str, 18: str, 19: str, 55: str}
df_origin = pd.read_csv('../data_new/a_marked_with_matched.csv', dtype=dtype_dict)
df_origin.head()

,CUST_ID,CUST_TYPE,CONF_DATE,CONF_TIME,CONF_SNO,BUSI_CODE,ACCO_ID,TRADE_ID,FUND_CODE,APP_DATE,...,PROVINCE,CITY,COUNTY,LAWID_CERT_VALID_DATE,CUST_ID_UN,BUSINESSSCOPE,CUST_NAME,MANAGE_RANGE,Key,Matched
0,369000460487,1,20240102,NaN,3690000011197,139,441*****7036,180621,000596,20231229,...,32.0,3201.0,320102.0,NaN,3.690005e+11,NaN,刘*,NaN,"(369000460487, '20240102')",False
1,369000450364,1,20240102,NaN,3690000011198,139,441*****4794,200424,001986,20231229,...,31.0,3101.0,310107.0,NaN,3.690005e+11,NaN,程*鹏,NaN,"(369000450364, '20240102')",False
2,369000610014,1,20240102,NaN,3690000011742,139,3M1*****9071,350017,005928,20231229,...,65.0,6501.0,650104.0,NaN,3.690006e+11,NaN,陈*,NaN,"(369000610014, '20240102')",False
3,369001110005,1,20240102,NaN,3690000292417,122,161*****0739,850007,004399,20231229,...,44.0,4403.0,440305.0,NaN,3.690011e+11,NaN,陈*,NaN,"(369001110005, '20240102')",False
4,369001110005,1,20240102,NaN,3690000292418,122,161*****0739,850007,004399,20231229,...,44.0,4403.0,440305.0,NaN,3.690011e+11,NaN,陈*,NaN,"(369001110005, '20240102')",False


In [3]:
value_counts = df_origin['CUST_ID'].value_counts()
print(value_counts)

value_counts2 = df_origin['TRADE_ID'].value_counts()
print(value_counts2)


CUST_ID
369001230019    2212
369000170047    2176
369000620008    2057
88448           1915
369001320010    1616
                ... 
369001030113       1
369001030089       1
369000980075       1
369001040088       1
369001050105       1
Name: count, Length: 1956, dtype: int64
TRADE_ID
99990000000171957    2073
33455                1915
980033               1815
1060011              1202
200591                928
                     ... 
0000001170138           1
0000001170136           1
0000001180152           1
0000000900077           1
0000000900011           1
Name: count, Length: 2583, dtype: int64


In [4]:
# 筛选列
df = df_origin[['CUST_ID', 'CUST_TYPE', 'CONF_DATE', 'BUSI_CODE',
             'FUND_CODE','CONF_AMTS','GENDER','BIRTH','TELL',
             'NET_CODE','RISK_LEV','COUNTY', 'Matched']]  # 双括号表示选择多个列
df.to_csv('../data/ziguan_select.csv',index=False)
df.head()



,CUST_ID,CUST_TYPE,CONF_DATE,BUSI_CODE,FUND_CODE,CONF_AMTS,GENDER,BIRTH,TELL,NET_CODE,RISK_LEV,COUNTY,Matched
0,369000460487,1,20240102,139,000596,50.00,2.0,19770726.0,139****9808,etrading,4.0,320102.0,False
1,369000450364,1,20240102,139,001986,10.00,1.0,19900106.0,138****8973,etrading,3.0,310107.0,False
2,369000610014,1,20240102,139,005928,1.00,2.0,19640306.0,136****5770,etrading,5.0,650104.0,False
3,369001110005,1,20240102,122,004399,95.77,2.0,19860711.0,186****7616,etrading,4.0,440305.0,False
4,369001110005,1,20240102,122,004399,23.94,2.0,19860711.0,186****7616,etrading,4.0,440305.0,False


In [5]:
# 计算每列的空值数量
null_counts = df.isnull().sum()

# 打印结果
print("每列的空值数量:")
print(null_counts)


每列的空值数量:
CUST_ID          0
CUST_TYPE        0
CONF_DATE        0
BUSI_CODE        0
FUND_CODE        0
CONF_AMTS        0
GENDER       27511
BIRTH        27511
TELL          7184
NET_CODE      6432
RISK_LEV      6432
COUNTY       27328
Matched          0
dtype: int64


In [6]:
# 创建一个交叉分析表，显示GENDER为空时其他列的空值情况
analysis = {}

for column in df.columns:
    if column != 'GENDER':
        # 计算当GENDER为空时，该列也为空的记录数
        both_null = df[(df['GENDER'].isnull()) & (df[column].isnull())].shape[0]
        # 计算当GENDER为空时，该列不为空的记录数
        gender_null_only = df[(df['GENDER'].isnull()) & (~df[column].isnull())].shape[0]
        
        analysis[column] = {
            'both_null': both_null,
            'gender_null_only': gender_null_only,
            'both_null_percentage': (both_null / df['GENDER'].isnull().sum()) * 100
        }

# 打印分析结果
print("{:<15} {:<15} {:<15} {:<15}".format('Column', 'Both Null', 'Only GENDER Null', 'Both Null %'))
for col, data in analysis.items():
    print("{:<15} {:<15} {:<15} {:<15.2f}".format(
        col, 
        data['both_null'], 
        data['gender_null_only'], 
        data['both_null_percentage']
    ))

Column          Both Null       Only GENDER Null Both Null %    
CUST_ID         0               27511           0.00           
CUST_TYPE       0               27511           0.00           
CONF_DATE       0               27511           0.00           
BUSI_CODE       0               27511           0.00           
FUND_CODE       0               27511           0.00           
CONF_AMTS       0               27511           0.00           
BIRTH           27511           0               100.00         
TELL            7184            20327           26.11          
NET_CODE        6432            21079           23.38          
RISK_LEV        6432            21079           23.38          
COUNTY          27328           183             99.33          
Matched         0               27511           0.00           


## 处理空值

In [7]:
# 查看原始数据行数
original_rows = len(df)
print(f"原始数据行数: {original_rows}")
# 删除CONF_AMTS列中小于20的行
# df_filtered = df[df['CONF_AMTS'] >= 20]  # 保留大于等于20的行
df_filtered = df
# 查看删除后的行数
filtered_rows = len(df_filtered)
print(f"删除CONF_AMTS<20后的行数: {filtered_rows}")
print(f"删除了 {original_rows - filtered_rows} 行")


原始数据行数: 54072
删除CONF_AMTS<20后的行数: 54072
删除了 0 行


In [8]:
# 计算每列的空值数量
null_counts = df_filtered.isnull().sum()

# 打印结果
print("每列的空值数量:")
print(null_counts)
df_filtered.head()

每列的空值数量:
CUST_ID          0
CUST_TYPE        0
CONF_DATE        0
BUSI_CODE        0
FUND_CODE        0
CONF_AMTS        0
GENDER       27511
BIRTH        27511
TELL          7184
NET_CODE      6432
RISK_LEV      6432
COUNTY       27328
Matched          0
dtype: int64


,CUST_ID,CUST_TYPE,CONF_DATE,BUSI_CODE,FUND_CODE,CONF_AMTS,GENDER,BIRTH,TELL,NET_CODE,RISK_LEV,COUNTY,Matched
0,369000460487,1,20240102,139,000596,50.00,2.0,19770726.0,139****9808,etrading,4.0,320102.0,False
1,369000450364,1,20240102,139,001986,10.00,1.0,19900106.0,138****8973,etrading,3.0,310107.0,False
2,369000610014,1,20240102,139,005928,1.00,2.0,19640306.0,136****5770,etrading,5.0,650104.0,False
3,369001110005,1,20240102,122,004399,95.77,2.0,19860711.0,186****7616,etrading,4.0,440305.0,False
4,369001110005,1,20240102,122,004399,23.94,2.0,19860711.0,186****7616,etrading,4.0,440305.0,False


In [9]:
# 计算每列的空值数量
null_counts = df_filtered.isnull().sum()

# 打印结果
print("每列的空值数量:")
print(null_counts)

每列的空值数量:
CUST_ID          0
CUST_TYPE        0
CONF_DATE        0
BUSI_CODE        0
FUND_CODE        0
CONF_AMTS        0
GENDER       27511
BIRTH        27511
TELL          7184
NET_CODE      6432
RISK_LEV      6432
COUNTY       27328
Matched          0
dtype: int64


In [10]:
df_filtered.to_csv('../data_new/ziguan_full.csv',index=False)